<a href="https://colab.research.google.com/github/kapumota/Actividades/blob/main/Caso-Refactorizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Refactorización con pruebas en Python


El caso de uso simple es el de una API de servicio a la que podemos acceder y que produce datos en formato JSON, es decir, una lista de elementos como la que se muestra aquí.


In [1]:
{
    "edad": 20,
    "apodo": "kapumota",
    "nombre": "Checha",
    "salario": "$943"
}

{'edad': 20, 'apodo': 'kapumota', 'nombre': 'Checha', 'salario': '$943'}

Escribimos una clase que calcula algunas estadísticas sobre los datos de entrada. Esta clase, llamada `DataStats`  proporciona un solo método `stats()`, cuyas entradas son los datos devueltos por el servicio (en formato JSON) y dos números enteros llamados `iedad` e `isalario`. 

El codigo es el siguiente:


In [7]:
%%writefile datastats.py 

import math
import json


class DataStats:
  def stats(self, data, iedad, isalario):
    incremento_edad_promedio = math.floor(sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio = math.floor( sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    incremento_anual_promedio = math.floor(incremento_salario_promedio/incremento_edad_promedio)

    salarios = [int(e['salario'][1:]) for e in data]
    limite = '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return json.dumps({
        'edad_promedio': math.floor(sum([e['edad'] for e in data])/len(data)),
        'salario_promedio': math.floor(sum( [int(e['salario'][1:]) for e in data])/len(data)),
        'incremento_anual_promedio': incremento_anual_promedio,
        'max_salario': max_salario,
        'min_salario': min_salario
        })

Overwriting datastats.py


**Pregunta:** ¿Puedes encontrar algunos problemas de la clase anterior?

In [ ]:
# Hay repetición de código
# No hay método Init
# Es dificil de depurar

#### 1 Prueba de los endpoints

Sean los siguientes datos:

In [2]:
test_data = [
    {
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
    },
    {
        "id": 2,
        "nombre": "Claudio",
        "apodo": "Avila",
        "edad": 49,
        "salario": "$67137"
    },
    {
        "id": 3,
        "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }
]

**Pregunta:** Llamando al método `stats()` con esa salida, con `iedad` establecido en `20` e `isalario` establecido en `20000`, cual es el resultado JSON.

In [3]:
# Tus respuestas
from datastats import DataStats
ds = DataStats()
ds.stats(test_data,20,20000)

'{"edad_promedio": 62, "salario_promedio": 55165, "incremento_anual_promedio": 837, "max_salario": [{"id": 3, "nombre": "Tomo", "apodo": "Frugs", "edad": 70, "salario": "$70472"}], "min_salario": [{"id": 1, "nombre": "Irene", "apodo": "Lara", "edad": 68, "salario": "$27888"}]}'

In [4]:
%%writefile test_datastats.py

import json

from datastats import DataStats


def test_json():
    test_data = [
    {
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
    },
    {
        "id": 2,
        "nombre": "Claudio",
        "apodo": "Avila",
        "edad": 49,
        "salario": "$67137"
    },
    {
        "id": 3,
        "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }
]
   
    ds = DataStats()

    assert ds.stats(test_data, 20, 20000) == json.dumps(
        {

        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
        }]
    }
        
)

Overwriting test_datastats.py


In [5]:
! pytest -v test_datastats.py

============================= test session starts =============================
platform win32 -- Python 3.11.1, pytest-7.3.1, pluggy-1.0.0 -- C:\Users\Ademar\AppData\Local\Programs\Python\Python311\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Ademar\OneDrive\Desktop\CC-3S2\Actividades\Caso-Refact-Py
plugins: cov-4.0.0
collecting ... collected 1 item

test_datastats.py::test_json PASSED                                      [100%]

============================== 1 passed in 0.04s ==============================


#### 2  Deshacerse del formato JSON


El método devuelve su salida en formato JSON, y mirando la clase es bastante evidente que la conversión la realiza  `json.dumps()`.

La estructura del código es la siguiente.

In [ ]:
class DataStats:
  def stats(self, data, iedad, isalario):
    [code_parte_1]

    return json.dumps({
        [code_parte_2]
        })

Donde obviamente `code_parte_2` depende de `code_parte_1`. La primera refactorización entonces sigue el siguiente procedimiento

1. Escribimos una prueba llamada `test__stats()` para un método `_stats()` que se supone que devolverá los datos como una estructura de Python. Podemos inferir esto último manualmente desde JSON o ejecutando `json.loads()` desde un shell de Python. La prueba falla.

2. Duplicamos el código del método `stats()` que produce los datos, colocándolo en el nuevo método `_stats()`. La prueba  debe pasar.

3. Eliminamos el código duplicado en `stats()` reemplazándolo con una llamada a `_stats()`.


Ahora el código de la clase se ve así:


In [9]:
%%writefile datastats.py 
import math
import json

class DataStats:
  
  def _stats(self, data, iedad, isalario):
    
    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio= math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    incremento_anual_promedio = math.floor(
            incremento_salario_promedio/incremento_edad_promedio)

    salarios = [int(e['salario'][1:]) for e in data]
    limite= '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return {
            'edad_promedio': math.floor(sum([e['edad'] for e in data])/len(data)),
            'salario_promedio': math.floor(sum(
                [int(e['salario'][1:]) for e in data])/len(data)),
            'incremento_anual_promedio': incremento_anual_promedio,
            'max_salario': max_salario,
            'min_salario': min_salario
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

Overwriting datastats.py


Y tenemos dos pruebas que comprueban la corrección de la misma.


In [10]:
%%writefile test_datastats.py

import json

from datastats import DataStats


def test_json():
    test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888" 
        },
       
       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

    ds = DataStats()

    assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )
        

def test__stats():
    test_data = [
        {
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
    },
    {
        "id": 2,
        "nombre": "Claudio",
        "apodo": "Avila",
        "edad": 49,
        "salario": "$67137"
    },
    {
        "id": 3,
        "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }
        
    ]

    ds = DataStats()

    assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
        
        }]
    }

Overwriting test_datastats.py


In [11]:
! pytest test_datastats.py

============================= test session starts =============================
platform win32 -- Python 3.11.1, pytest-7.3.1, pluggy-1.0.0
rootdir: c:\Users\Ademar\OneDrive\Desktop\CC-3S2\Actividades
plugins: cov-4.0.0
collected 2 items

test_datastats.py ..                                                     [100%]

============================== 2 passed in 0.05s ==============================


#### 3 Refactorización de las pruebas


Está bastante claro que la lista de diccionarios `test_data` se usará en cada prueba que realicemos, por lo que ya es hora de que la traslademos a una variable global. 
También podríamos mover los datos de salida a una variable global.

El conjunto de pruebas ahora parece:

In [1]:
%%writefile test_datastats.py

import json

from datastats import DataStats


test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888" 
        },
       
       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

def test_json():
  
  ds = DataStats()

  assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )
        

def test__stats():

   ds = DataStats()
   
   assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
        
        }]
    }

Overwriting test_datastats.py


In [2]:
! pytest test_datastats.py

============================= test session starts =============================
platform win32 -- Python 3.11.1, pytest-7.3.1, pluggy-1.0.0
rootdir: c:\Users\Ademar\OneDrive\Desktop\CC-3S2\Actividades\Caso-Refact-Py
plugins: cov-4.0.0
collected 2 items

test_datastats.py ..                                                     [100%]

============================== 2 passed in 0.04s ==============================


#### 4 Aislamiento del algoritmo de edad promedio

Aislar características independientes es un objetivo clave del diseño de software. Por lo tanto, la refactorización tendrá como objetivo desentrañar el código dividiéndolo en pequeñas funciones separadas.


Para aislar algún código, lo primero que debe hacer es duplicarlo, colocándolo en un método dedicado. Como estamos refactorizando con pruebas, lo primero es escribir una prueba para este método.


In [3]:
%%writefile datastats.py 
import math
import json

class DataStats:

  def _edad_promedio(self, data):
    return math.floor(sum([e['edad'] for e in data])/len(data))
  
  def _stats(self, data, iedad, isalario):
    
    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio= math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    incremento_anual_promedio = math.floor(
            incremento_salario_promedio/incremento_edad_promedio)

    salarios = [int(e['salario'][1:]) for e in data]
    limite= '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': math.floor(sum(
                [int(e['salario'][1:]) for e in data])/len(data)),
            'incremento_anual_promedio': incremento_anual_promedio,
            'max_salario': max_salario,
            'min_salario': min_salario
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

Overwriting datastats.py


In [4]:
%%writefile test_datastats.py

import json

from datastats import DataStats


test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888" 
        },
       
       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

def test_json():
  
  ds = DataStats()

  assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )
        

def test__stats():

   ds = DataStats()
   
   assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
        
        }]
    }

def test__edad_promedio():
  ds = DataStats()
  
  assert ds._edad_promedio(test_data) == 62

Overwriting test_datastats.py


In [5]:
! pytest test_datastats.py

============================= test session starts =============================
platform win32 -- Python 3.11.1, pytest-7.3.1, pluggy-1.0.0
rootdir: c:\Users\Ademar\OneDrive\Desktop\CC-3S2\Actividades\Caso-Refact-Py
plugins: cov-4.0.0
collected 3 items

test_datastats.py ...                                                    [100%]

============================== 3 passed in 0.05s ==============================


#### 5 Aíslamiento el algoritmo de salario promedio


In [10]:
%%writefile datastats.py 
import math
import json

class DataStats:
  def _salario_promedio(self, data):
    return math.floor(sum([int(e['salario'][1:]) for e in data])/len(data))

  def _edad_promedio(self, data):
    return math.floor(sum([e['edad'] for e in data])/len(data))
  
  def _stats(self, data, iedad, isalario):
    
    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio= math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    incremento_anual_promedio = math.floor(
            incremento_salario_promedio/incremento_edad_promedio)

    salarios = [int(e['salario'][1:]) for e in data]
    limite= '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': self._salario_promedio(data),
            'incremento_anual_promedio': incremento_anual_promedio,
            'max_salario': max_salario,
            'min_salario': min_salario
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

Overwriting datastats.py


In [11]:
%%writefile test_datastats.py

import json

from datastats import DataStats


test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888" 
        },
       
       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

def test_json():
  
  ds = DataStats()

  assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )
        

def test__stats():

   ds = DataStats()
   
   assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
        
        }]
    }

def test__edad_promedio():
  ds = DataStats()
  
  assert ds._edad_promedio(test_data) == 62

def test__salario_promedio():
  ds = DataStats()
  assert ds._salario_promedio(test_data) == 55165

Overwriting test_datastats.py


In [12]:
! pytest test_datastats.py

============================= test session starts =============================
platform win32 -- Python 3.11.1, pytest-7.3.1, pluggy-1.0.0
rootdir: c:\Users\Ademar\OneDrive\Desktop\CC-3S2\Actividades\Caso-Refact-Py
plugins: cov-4.0.0
collected 4 items

test_datastats.py ....                                                   [100%]

============================== 4 passed in 0.04s ==============================


#### 6 Aislamiento del algoritmo de aumento anual promedio

Para el incremento promedio anual del salario tenemos una nueva prueba:

In [ ]:
#def test__incremento_anual_promedio():
#  ds = DataStats()
#  assert ds._incremento_anual_promedio(test_data, 20, 20000) == 837

Un nuevo método que pasa la prueba:

In [ ]:
#def _incremento_anual_promedio(self, data, iedad, isalario):
#  incremento_anual_promedio = math.floor(
#            sum([e['edad'] for e in data])/len(data)) - iedad
#  incremento_salario_promedio = math.floor(
#            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

#  return math.floor(incremento_salario_promedio/incremento_edad_promedio)

Y una nueva versión del método `_stats()`:

In [ ]:
#def _stats(self, data, iedad, isalario):
#  salarios = [int(e['salario'][1:]) for e in data]
#  limite= '$' + str(max(salarios))
#
#  max_salario = [e for e in data if e['salario'] == limite]
#
#  salarios = [int(d['salario'][1:]) for d in data]
#  min_salario = [e for e in data if e['salario'] ==
#                      '${}'.format(str(min(salarios)))]
#
#  return {
#        'edad_promedio': self._avg_age(data),
#        'salario_promedio':  self._salario_promedio(data),
#        'incremento_anual_promedio': self._incremento_anual_promedio (
#                data, iedad, isalario),
#        'max_salario': max_salario,
#        'min_salario': min_salario
#        }

Con todo esto tenemos:

In [13]:
%%writefile datastats.py 
import math
import json

class DataStats:
  
  def _salario_promedio(self, data):
    return math.floor(sum([int(e['salario'][1:]) for e in data])/len(data))

  def _edad_promedio(self, data):
    return math.floor(sum([e['edad'] for e in data])/len(data))

  def _incremento_anual_promedio(self, data, iedad, isalario):
    
    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio = math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    return math.floor(incremento_salario_promedio/incremento_edad_promedio)
  
  def _stats(self, data, iedad, isalario):
    salarios = [int(e['salario'][1:]) for e in data]
    limite= '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': self._salario_promedio(data),
            'incremento_anual_promedio': self._incremento_anual_promedio(
                data, iedad, isalario),
            'max_salario': max_salario,
            'min_salario': min_salario
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

Overwriting datastats.py


In [14]:
%%writefile test_datastats.py

import json

from datastats import DataStats


test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888" 
        },
       
       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

def test_json():
  
  ds = DataStats()

  assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )
        

def test__stats():

   ds = DataStats()
   
   assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
        
        }]
    }

def test__edad_promedio():
  ds = DataStats()
  
  assert ds._edad_promedio(test_data) == 62

def test__salario_promedio():
  ds = DataStats()
  assert ds._salario_promedio(test_data) == 55165

def test__incremento_anual_promedio():
  ds = DataStats()

  assert ds._incremento_anual_promedio(test_data, 20, 20000) == 837


Overwriting test_datastats.py


In [15]:
! pytest test_datastats.py

============================= test session starts =============================
platform win32 -- Python 3.11.1, pytest-7.3.1, pluggy-1.0.0
rootdir: c:\Users\Ademar\OneDrive\Desktop\CC-3S2\Actividades\Caso-Refact-Py
plugins: cov-4.0.0
collected 5 items

test_datastats.py .....                                                  [100%]

============================== 5 passed in 0.05s ==============================


### 7 - Aislamiento los algoritmos de salario máximo y mínimo


Para este caso identificamos las nuevas pruebas.

In [16]:
%%writefile datastats.py 
import math
import json

class DataStats:
  
  def _salario_promedio(self, data):
    return math.floor(sum([int(e['salario'][1:]) for e in data])/len(data))

  def _edad_promedio(self, data):
    return math.floor(sum([e['edad'] for e in data])/len(data))

  def _incremento_anual_promedio(self, data, iedad, isalario):
    
    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio = math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    return math.floor(incremento_salario_promedio/incremento_edad_promedio)
  
  def _max_salario(self, data):
    salarios = [int(e['salario'][1:]) for e in data]
    limite = '$' + str(max(salarios))

    return [e for e in data if e['salario'] == limite]

  def _min_salario(self, data):
    salarios = [int(d['salario'][1:]) for d in data]
    return [e for e in data if e['salario'] ==
                '${}'.format(str(min(salarios)))]
  
  
  def _stats(self, data, iedad, isalario):
    return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': self._salario_promedio(data),
            'incremento_anual_promedio': self._incremento_anual_promedio(
                data, iedad, isalario),
            'max_salario': self._max_salario(data),
            'min_salario': self._min_salario(data)
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

Overwriting datastats.py


In [17]:
%%writefile test_datastats.py

import json

from datastats import DataStats

test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888" 
        },
       
       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

def test_json():
  
  ds = DataStats()

  assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )
        

def test__stats():

   ds = DataStats()
   
   assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
        
        }]
    }

def test__edad_promedio():
  ds = DataStats()
  assert ds._edad_promedio(test_data) == 62

def test__salario_promedio():
  ds = DataStats()
  assert ds._salario_promedio(test_data) == 55165

def test__incremento_anual_promedio():
  ds = DataStats()
  assert ds._incremento_anual_promedio(test_data, 20, 20000) == 837


def test__max_salario():  
  ds = DataStats()
  
  assert ds._max_salario(test_data) == [{
        "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
    }]

def test__min_salario():
  ds = DataStats()
  assert ds._min_salario(test_data) ==[{
      "id": 1,
      "nombre": "Irene",
      "apodo": "Lara",
      "edad": 68,
      "salario": "$27888"
       }]

Overwriting test_datastats.py


In [18]:
! pytest -v test_datastats.py

============================= test session starts =============================
platform win32 -- Python 3.11.1, pytest-7.3.1, pluggy-1.0.0 -- C:\Users\Ademar\AppData\Local\Programs\Python\Python311\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Ademar\OneDrive\Desktop\CC-3S2\Actividades\Caso-Refact-Py
plugins: cov-4.0.0
collecting ... collected 7 items

test_datastats.py::test_json PASSED                                      [ 14%]
test_datastats.py::test__stats PASSED                                    [ 28%]
test_datastats.py::test__edad_promedio PASSED                            [ 42%]
test_datastats.py::test__salario_promedio PASSED                         [ 57%]
test_datastats.py::test__incremento_anual_promedio PASSED                [ 71%]
test_datastats.py::test__max_salario PASSED                              [ 85%]
test_datastats.py::test__min_salario PASSED                              [100%]

============================== 7 passed in 0.05s ===========================

In [19]:
! pip install coverage
! pip install pytest-cov

In [20]:
!pytest --cov=datastats datastats.py

============================= test session starts =============================
platform win32 -- Python 3.11.1, pytest-7.3.1, pluggy-1.0.0
rootdir: c:\Users\Ademar\OneDrive\Desktop\CC-3S2\Actividades\Caso-Refact-Py
plugins: cov-4.0.0
collected 0 items


---------- coverage: platform win32, python 3.11.1-final-0 -----------
Name           Stmts   Miss  Cover
----------------------------------
datastats.py      22     12    45%
----------------------------------
TOTAL             22     12    45%

============================ no tests ran in 0.07s ============================


#### 8 Reducción de la duplicación de código

Del código anterior los dos métodos `_max_salario()` y `_min_salario()` comparten una gran cantidad de código, aunque el segundo es más conciso.


In [21]:
def _max_salario(self, data):
  salarios = [int(e['salario'][1:]) for e in data]
  limite = '$' + str(max(salarios))

  return [e for e in data if e['salario'] == limite]

def _min_salario(self, data):
  salarios = [int(d['salario'][1:]) for d in data]
  return [e for e in data if e['salario'] ==
                '$'.format(str(min(salarios)))]

Veamos el caso de la variable `limite` en la segunda función. Tan pronto como se cambie algo, ejecuta las pruebas para verificar que el comportamiento externo no haya cambiado.

In [22]:
def _max_salario(self, data):
  salarios = [int(e['salario'][1:]) for e in data]
  limite = '$' + str(max(salarios))

  return [e for e in data if e['salario'] == limite]

def _min_salario(self, data):
  salarios = [int(d['salario'][1:]) for d in data]
  limite = '$'.format(str(min(salarios)))
    
  return [e for e in data if e['salario'] == limite]

Ahora las dos funciones son las mismas pero para las funciones `min()` y `max()`. Todavía usan diferentes nombres de variables y diferentes códigos para formatear el limite, por lo que una primera acción es igualarlos, copiando el código de `_min_salario()` a `_max_salario()` y cambiando `min()` a `max()`-


In [23]:
def _max_salario(self, data):
  salarios = [int(e['salario'][1:]) for d in data]
  limite = '$'.format(str(max(salarios)))

  return [e for e in data if e['salario'] == limite]

def _min_salario(self, data):
  salarios = [int(d['salario'][1:]) for d in data]
  limite = '$'.format(str(min(salarios)))
    
  return [e for e in data if e['salario'] == limite]

Creamos la función `_select_salario()` que duplique ese código y acepte una función en lugar de `min()` o `max()`. 

In [25]:
def _select_salario(self, data, func):
  salarios = [int(d['salario'][1:]) for d in data]
  limite = '${}'.format(str(func(salarios)))

  return [e for e in data if e['salario'] == limite]

def _max_salario(self, data):
  return self._select_salario(data, max)

def _min_salario(self, data):
  return self._select_salario(data, min)

Hay una duplicación de código entre `_salario_promedio()` y `_select_salario()`.


In [26]:
def _salario_promedio(self, data):
  return math.floor(sum([int(e['salario'][1:]) for e in data])/len(data))

In [27]:
def _select_salario(self, data, func):
  salarios = [int(d['salario'][1:]) for d in data]

Extraemos el algoritmo común en un método llamado `_salarios()`, para ello primero escribimos la prueba.

In [28]:
def test_salarios():
  ds = DataStats()
  assert ds._salarios(test_data) == [27888, 67137, 70472]

Implementamos el método.

In [29]:
def _salarios(self, data):
  return [int(d['salario'][1:]) for d in data]

Reemplazamos el código duplicado con una llamada al nuevo método.

In [30]:
def _salarios(self, data):
  return [int(d['salario'][1:]) for d in data]

In [31]:
def _select_salario(self, data, func):
  limite= '${}'.format(str(func(self._salarios(data))))
  return [e for e in data if e['salario'] == limite]

Veamos el código de `_incremento_anual_promedio()`. ¿Cuál es el problema del código?

In [33]:
import math
def _incremento_anual_promedio(self, data, iedad, isalario):
  incremento_edad_promedio = math.floor(
      sum([e['edad'] for e in data])/len(data)) - iedad
    
  incremento_salario_promedio = math.floor(
      sum(self._salarios(data))/len(data)) - isalario
        
  return math.floor(incremento_salario_promedio/incremento_edad_promedio)

El código de `_incremento_anual_promedio()` contiene el código de los métodos `_edad_promedio()` y `_salario_promedio()`, por lo que vale la pena reemplazarlo con dos llamadas. 

In [34]:
import math
def _incremento_anual_promedio(self, data, iedad, isalario):
  incremento_edad_promedio= self._edad_promedio(data) - iedad
  incremento_salario_promedio = self._salario_promedio(data) - isalario

  return math.floor(incremento_salario_promedio/incremento_edad_promedio)

In [ ]:
%%writefile datastats.py 
import math
import json

class DataStats:
  def _max_salario(self, data):
    return self._select_salario(data, max)

  def _min_salario(self, data):
    return self._select_salario(data, min)

  def _salarios(self, data):
  return [int(d['salario'][1:]) for d in data]
  
  
  def _stats(self, data, iedad, isalario):
    return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': self._salario_promedio(data),
            'incremento_anual_promedio': self._incremento_anual_promedio(
                data, iedad, isalario),
            'max_salario': self._max_salario(data),
            'min_salario': self._min_salario(data)
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

#### 9 Refactorización avanzada

El objetivo es cambiar la clase actual para que coincida con la nueva API y luego crear una clase que envuelva la primera y proporcione la API anterior. . La estrategia no es tan diferente de lo que hicimos anteriormente, solo que esta vez trataremos con clases en lugar de métodos. Con un estupendo esfuerzo de mi imaginación nombré a la nueva clase `NDataStats`. 

Lo primero, como sucede muy a menudo con la refactorización, es duplicar el código, y cuando insertamos código nuevo necesitamos tener pruebas que lo justifiquen. 

Las pruebas serán las mismas que antes, ya que la nueva clase proporcionará las mismas funcionalidades que la anterior, así que solo crea un nuevo archivo, llamado `test_ndatastats.py` y allí se coloca la primera prueba `test_init()`.

In [ ]:
%%writefile test_ndatastats.py
import json

from datastats import NDataStats
test_data = [
    {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888" 
        },
       
       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    
]

def test_init():
  ds = NDataStats(test_data)
  assert ds.data == test_data

**Pregunta:** Escribe la nueva clase `NdataStats` en el archivo `datastats.py` y comprueba si la prueba anterior pasa.

In [ ]:
# Tu respuesta.

Ahora iniciamos un proceso iterativo:

- Copia una de las pruebas de `DataStats` y adapta  a `NDataStats`
- Copia algo de código de `DataStats` a `NDataStats`, adaptándolo a la nueva API y haciendo pasar una prueba.

En este punto, eliminar iterativamente métodos de `DataStats` y reemplazarlos con una llamada a `NDataStats` es una exageración.

Un ejemplo de las pruebas resultantes para `NDataStats` es el siguiente:

In [ ]:
#def test_edad():
#  ds = NDataStats(test_data)
#   assert ds._edades() == [68, 49, 70]

Y el código que pasa la prueba para este caso es:

In [ ]:
#def _edades(self):
#  return [d['edad'] for d in self.data]

Los métodos como `_edades()` ya no requieren un parámetro de entrada y se pueden convertir convertirlos en propiedades, cambiando las pruebas con `@property`.

Es hora de reemplazar los métodos de `DataStats` con llamadas a `NDataStats`. Podríamos hacerlo método por método, pero en realidad lo único que realmente necesitamos es reemplazar `stats()`. Así que el nuevo código es:

In [ ]:
#class DataStats:
#  def stats(self, data, iedad, isalario):
#    nds = NDataStats(data)
#    return nds.stats(iedad, isalario)

**Pregunta:** Edita los archivos necesarios para ejecutar la prueba. ¿Cuál es el código de cobertura para este ejemplo.

In [ ]:
# Tu respuesta

**Pregunta:** La refactorización es un proceso iterativo, a menudo sucederá que crees que hiciste todo lo posible, solo para descubrir más tarde que te perdiste algo. En este caso, el paso faltante es una pequeña duplicación de código.

Las dos funciones comparten la misma lógica, por lo que definitivamente podemos aislar esto  y llamar al código común en cada función.

In [ ]:
#def _salario_promedio(self):
#  return math.floor(sum(self._salarios)/len(self.data))

#def _edad_promedio(self):
#  return math.floor(sum(self._edades)/len(self.data))

In [ ]:
#def _promedio_floor(self, suma_de_numberos):
#  return math.floor(suma_de_numeros / len(self.data))

#def _salario_promedio(self):
#  return self._promedio_floor(sum(self._salarios))

#def _edad_promedio(self):
#  return self._promedio_floor(sum(self._edades))

Edita los archivos necesarios para ejecutar la prueba. ¿Cuál es el código de cobertura para este ejemplo.

In [ ]:
## Tu respuesta.

**Referencias**

- [Why do developers hate code coverage? And why they should not hate it!](https://www.effective-software-testing.com/why-do-developers-hate-code-coverage)
- [Do unit tests make refactoring harder?](https://www.effective-software-testing.com/do-unit-tests-make-refactoring-harder).

In [ ]:
class NDataStats:
  def stats(self, data, iedad, isalario):
    incremento_edad_promedio = math.floor(sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio = math.floor( sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    incremento_anual_promedio = math.floor(incremento_salario_promedio/incremento_edad_promedio)

    salarios = [int(e['salario'][1:]) for e in data]
    limite = '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return json.dumps({
        'edad_promedio': math.floor(sum([e['edad'] for e in data])/len(data)),
        'salario_promedio': math.floor(sum( [int(e['salario'][1:]) for e in data])/len(data)),
        'incremento_anual_promedio': incremento_anual_promedio,
        'max_salario': max_salario,
        'min_salario': min_salario
        })